In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import os
import sys

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from state.board import Board, Color
from machine_learning.model import ChessNet
from machine_learning.utils import board_to_tensor, get_material_score

In [17]:
# --- CẤU HÌNH ---
EPISODES = 250       # Số ván tự chơi (Self-play)
MAX_MOVES = 500      # Giới hạn số nước đi để tránh loop vô tận
EPSILON_START = 0.9  # Tỉ lệ đi ngẫu nhiên ban đầu (khám phá)
EPSILON_END = 0.1    # Tỉ lệ đi ngẫu nhiên lúc sau (tối ưu)
EPSILON_DECAY = 0.995
LEARNING_RATE = 0.001
BATCH_SIZE = 128

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
def select_move_epsilon(model, board, epsilon):
    """Chọn nước đi: Epsilon% ngẫu nhiên, (1-Epsilon)% theo Model"""
    next_states = board.generate_next_states()
    if not next_states: return None

    # Khám phá (Exploration): Đi bừa để học cái mới
    if random.random() < epsilon:
        return random.choice(next_states)

    # Khai thác (Exploitation): Đi nước tốt nhất theo Model hiện tại
    best_state = None
    best_score = -float('inf') if board.turn == Color.WHITE else float('inf')
    
    # Batch processing để nhanh hơn
    tensor_list = []
    for st in next_states:
        tensor_list.append(board_to_tensor(st.board).numpy())
    
    batch_tensors = torch.tensor(np.array(tensor_list)).to(device)
    
    with torch.no_grad():
        scores = model(batch_tensors).cpu().numpy().flatten()
        
    if board.turn == Color.WHITE:
        idx = np.argmax(scores)
    else:
        idx = np.argmin(scores)
        
    return next_states[idx]

In [19]:
def train_self_play(episodes, max_moves, epsilon_start, epsilon_end, epsilon_decay, learning_rate, batch_size):
    print(f"--- Bắt đầu Training Self-Play trên {device} ---")
    
    model = ChessNet().to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    
    epsilon = epsilon_start
    memory = [] # Lưu trữ dữ liệu: (board_tensor, target_value)

    for episode in range(1, episodes + 1):
        board = Board()
        game_history = [] # Lưu các trạng thái trong ván này
        
        # --- GIAI ĐOẠN 1: TỰ CHƠI (SELF-PLAY) ---
        winner = 0 # 0: Hòa, 1: Trắng, -1: Đen
        moves_count = 0
        
        while moves_count < max_moves:
            # Kiểm tra kết thúc game
            if not board._has_legal_moves_for(board.turn):
                if board.is_in_check(board.turn):
                    winner = -1 if board.turn == Color.WHITE else 1
                else:
                    winner = 0 # Hòa stalemate
                break
                
            # Bot chọn nước đi
            next_state = select_move_epsilon(model, board, epsilon)
            if next_state is None: break # Hòa/Lỗi
            
            # Lưu trạng thái vào lịch sử ván đấu
            # Note: Lưu tensor của bàn cờ TRƯỚC khi đi hay SAU khi đi?
            # Với Value Network, ta đánh giá thế cờ HIỆN TẠI.
            game_history.append(board_to_tensor(next_state.board))
            
            board = next_state.board
            moves_count += 1
            
        # Giảm epsilon dần dần
        epsilon = max(epsilon_end, epsilon * epsilon_decay)

        # --- GIAI ĐOẠN 2: TẠO DỮ LIỆU TRAINING (REWARD ASSIGNMENT) ---
        # Gán nhãn cho toàn bộ nước đi trong ván
        # Nếu Trắng thắng (1): Mọi thế cờ dẫn đến kết quả này đều có xu hướng = 1
        # Nhưng để thông minh hơn, ta cộng thêm điểm Material (heuristic) để dẫn hướng ban đầu
        
        for state_tensor in game_history:
            # Reward shaping: Kết hợp Kết quả ván cờ + Lợi thế vật chất
            # target = (Kết quả thực tế * 0.7) + (Điểm vật chất quy đổi * 0.3)
            # Điều này giúp bot không bị "mù" khi chưa thắng ván nào
            
            # Tính điểm vật chất đơn giản (-1 đến 1)
            # Giả sử max material diff là 20
            # material_val = get_material_score(...) / 20.0 
            # Tuy nhiên để đơn giản, ta dùng Pure Monte Carlo trước:
            
            target = float(winner) 
            
            # Lưu vào bộ nhớ chung
            memory.append((state_tensor, target))

        # --- GIAI ĐOẠN 3: HUẤN LUYỆN (TRAINING) ---
        # Chỉ train khi đủ dữ liệu hoặc hết ván
        if len(memory) > batch_size:
            # Lấy ngẫu nhiên batch để học (Experience Replay)
            batch = random.sample(memory, batch_size)
            states_b, targets_b = zip(*batch)
            
            states_tensor = torch.stack(states_b).to(device)
            targets_tensor = torch.tensor(targets_b, dtype=torch.float32).unsqueeze(1).to(device)
            
            optimizer.zero_grad()
            outputs = model(states_tensor)
            loss = criterion(outputs, targets_tensor)
            loss.backward()
            optimizer.step()
            
            # Giữ bộ nhớ không quá lớn (quên bớt cái cũ)
            if len(memory) > 5000:
                memory = memory[-5000:]
                
        # Log kết quả
        if episode % 10 == 0:
            result_str = "Hòa"
            if winner == 1: result_str = "Trắng Thắng"
            if winner == -1: result_str = "Đen Thắng"
            print(f"Ep {episode}: {result_str} (Moves: {moves_count}, Eps: {epsilon:.2f}, Loss: {loss.item():.4f})")
            
        # Lưu model định kỳ
        if episode % 50 == 0:
            torch.save(model.state_dict(), "chess_model.pth")

    print("--- Hoàn tất Training RL ---")
    torch.save(model.state_dict(), "chess_model.pth") 


In [20]:
train_self_play(EPISODES, MAX_MOVES, EPSILON_START, EPSILON_END, EPSILON_DECAY, LEARNING_RATE, BATCH_SIZE)

--- Bắt đầu Training Self-Play trên cuda ---
Ep 10: Hòa (Moves: 500, Eps: 0.86, Loss: 0.1853)
Ep 20: Hòa (Moves: 500, Eps: 0.81, Loss: 0.1465)
Ep 30: Hòa (Moves: 500, Eps: 0.77, Loss: 0.0939)
Ep 40: Hòa (Moves: 500, Eps: 0.74, Loss: 0.0810)
Ep 50: Đen Thắng (Moves: 66, Eps: 0.70, Loss: 0.1221)
Ep 60: Trắng Thắng (Moves: 199, Eps: 0.67, Loss: 0.0772)
Ep 70: Hòa (Moves: 500, Eps: 0.63, Loss: 0.0834)
Ep 80: Hòa (Moves: 500, Eps: 0.60, Loss: 0.1248)
Ep 90: Hòa (Moves: 500, Eps: 0.57, Loss: 0.0029)
Ep 100: Hòa (Moves: 500, Eps: 0.55, Loss: 0.0020)
Ep 110: Hòa (Moves: 500, Eps: 0.52, Loss: 0.0942)
Ep 120: Hòa (Moves: 500, Eps: 0.49, Loss: 0.1643)
Ep 130: Hòa (Moves: 500, Eps: 0.47, Loss: 0.2048)
Ep 140: Trắng Thắng (Moves: 107, Eps: 0.45, Loss: 0.1346)
Ep 150: Hòa (Moves: 500, Eps: 0.42, Loss: 0.0619)
Ep 160: Hòa (Moves: 500, Eps: 0.40, Loss: 0.0323)
Ep 170: Hòa (Moves: 500, Eps: 0.38, Loss: 0.0660)
Ep 180: Hòa (Moves: 500, Eps: 0.37, Loss: 0.0318)
Ep 190: Hòa (Moves: 132, Eps: 0.35, Loss: 0